In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
def collect_accuracies(path, setup_group, verbose=1):
    report = {}
    setup_paths = []
    for filename in glob.iglob(path + '**/accuracy_report*', recursive=True):
        name_split = filename.split('/')
        if setup_group in name_split[-2]:
            clf = name_split[-1].split('_')[-1].split('.')[0]
            setup_paths.append(filename.replace(name_split[-1], ''))
            setup = clf + '_' + name_split[-2].replace(setup_group, '')
            if verbose:
                print('Setup: %s' % (setup))
            report[setup] = get_accuracies(filename, verbose)
    return report, setup_paths

def get_accuracies(path, verbose=1):
    accs = {}
    with open(path, "r") as f:
        line = f.readline()
    for l in line.split('| '):
        sname = l.split(' ')[0]
        acc = l.split(' ')[2]
        if verbose:
            print('Set: %s | Accuracy: %s' % (sname, acc))
        accs[sname] = acc
    return accs

In [ ]:
path = "../../models/" 
setup = "anomaly_new/"
path = os.path.join(path, setup)
setup_group = "strides_5_length_10_"
accuracy_report, setup_paths = collect_accuracies(path, setup_group, verbose=1)

In [ ]:
def autolabel(ax, rects, norm=True, xpos='center'):
    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0, 'right': 1, 'left': -1}

    for rect in rects:
        height = rect.get_height()
        ax.annotate('%.2f' % (height*100 if norm else height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(offset[xpos]*3, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha=ha[xpos], va='bottom')

def plot_accuracies(acc_rep, measurement='Accuracy', fname = 'Accuracies', save = None):
    sl = len(acc_rep.keys())
    sl_0 = acc_rep[list(acc_rep.keys())[0]]
    sel = len(list(sl_0.keys()))
    
    accs = np.zeros((sel, sl))
    for s, setup in enumerate(acc_rep.keys()):
        for se, setname in enumerate(acc_rep[setup].keys()):
            accs[se, s] = acc_rep[setup][setname]

    ind = np.arange(sel)
    width = 0.15
    offset = (accs.shape[1] / 2) * width
    if accs.shape[1] % 2 > 0:
        offset -= width/2
    fig, ax = plt.subplots(figsize=(20,4))
    for s in range(sl):
        r = ax.bar(ind + s * width - offset, accs[:,s], width, label=list(acc_rep.keys())[s])
        if measurement == 'Loss':
            autolabel(ax, r, False)
        else:
            autolabel(ax, r)
    
    ax.set_ylabel(measurement)
    title = ax.set_title(measurement + ' grouped by dataset')
    ax.set_xticks(np.arange(accs.shape[0]))
    ax.set_xticklabels(list(sl_0.keys()))
    ax.legend()

    plt.ylim(np.min(accs[se]*0.9))
    title.set_position([.5, 1.2])
    plt.legend(bbox_to_anchor=(0,1.02,1,0.2), loc="lower left",
                mode="expand", borderaxespad=0.0, ncol=5)
    fig.tight_layout()
    
    if not save is None:
        if not os.path.exists(save):
            os.makedirs(save)
        plt.savefig(os.path.join(save, fname), dpi=300)

    plt.show()

In [ ]:
image_path = '../../images/Setup_Comparison/'
image_path = os.path.join(image_path, setup, setup_group)
plot_accuracies(accuracy_report, save=image_path)

In [ ]:
def get_setup_params(key):
    splits = key.split('_')
    z, a, n = splits[0][-1], splits[1][-1], splits[2][-1]
    return np.array([z, a, n], dtype=int)

In [ ]:
from copy import deepcopy

from modules.data_processor import generate_data
from modules.data_generator import DataGenerator
from modules.patch_generator import get_data_patch_stats, get_generator_id_list
from modules.model import create_model
from modules.model_trainer import train_descriptive

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

import sys
sys.path.append("../")
%load_ext autoreload
%autoreload 2

In [ ]:
data_path = '../../data/anomaly_new/anomaly_dataset.pickle'
trainX, trainY, valX, valY, testX, testY, classes, seqlen, channel = generate_data(data_path, create_val=True, verbose=1)
trainLen, valLen, testLen = trainX.shape[0], valX.shape[0], testX.shape[0]

# [Stride, Length]
config = [[5,10]]

l1_acc = deepcopy(accuracy_report)
l1_loss = deepcopy(accuracy_report)
models = []

for i, key in enumerate(list(accuracy_report.keys())):
    print('Evaluate:', key)
    # define for each run
    zero, attach, notemp = get_setup_params(key)
    params = {'dim': [seqlen, channel], 'batch_size': 1024, 'config': config,
            'zero': zero, 'attach': attach, 'notemp': notemp, 'shuffle': False}

    # Generators
    trainIds = get_generator_id_list(trainLen, seqlen, config)
    train_generator = DataGenerator(trainIds, trainX, trainY, **params)
    valIds = get_generator_id_list(valLen, seqlen, config)
    val_generator = DataGenerator(valIds, valX, valY, **params)
    testIds = get_generator_id_list(testLen, seqlen, config)
    test_generator = DataGenerator(testIds, testX, testY, **params)

    input_shape = trainX.shape[1:]
    if attach:
        input_shape = list(input_shape)
        input_shape[-1] +=1
        input_shape = tuple(input_shape)
    patch_model_path = os.path.join(setup_paths[i], 'patch_classifier.h5')
    patch_model = tf.keras.models.load_model(patch_model_path)
    models.append(patch_model)

    l1_loss[key]['Train'], l1_acc[key]['Train'] = patch_model.evaluate(train_generator)
    l1_loss[key]['Val'], l1_acc[key]['Val'] = patch_model.evaluate(val_generator)
    l1_loss[key]['Test'], l1_acc[key]['Test'] = patch_model.evaluate(test_generator)

In [ ]:
plot_accuracies(l1_acc, measurement='Accuracy', fname='Accuracies_Level_1', save=image_path)
plot_accuracies(l1_loss, measurement='Loss')